In [22]:
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.management import *
from arcpy.conversion import *
from arcpy.da import *
from arcpy.sa import *

In [23]:
# 辅助函数
# 栅格转面
def raster_to_polygon(in_raster, out_polygon, field_name):
    """栅格转面"""
    in_raster = arcpy.sa.Int(in_raster)
    output_coordinate_system = arcpy.Describe(in_raster).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=output_coordinate_system):
        arcpy.conversion.RasterToPolygon(in_raster, out_polygon, "SIMPLIFY", f"{field_name}", "SINGLE_OUTER_PART", None)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [24]:
# 构建转换字典
table_path = r"F:\cache_data\frequency_filter\ky\scaler_csv\ky_stander_filter_all_type.csv"

In [25]:
df = pd.read_csv(table_path)

In [26]:
# 对每个唯一的土种编号生成一个字典
def aggregate_rows(group):
    records = group[['TL', 'YL', 'TS']].to_dict('records')
    # 假设每个土种编号下的记录是相同的，这里我们取第一个记录
    return records[0] if records else {}

# 使用groupby按照`土种编号`聚合，并应用自定义函数
soil_dict = df.groupby('TZ').apply(aggregate_rows).to_dict()

In [27]:
tz_dict = {0: ['中层次生黄色石灰土'], 1: ['中层淋溶黄色石灰土'], 2: ['中层硅质黄壤'], 3: ['中层硅铁质黄壤'], 4: ['中层硅铁质黄壤性土'], 5: ['中层硅铁质黄棕壤'], 6: ['中层硅铝质黄壤'], 7: ['中层硅铝质黄壤性土'], 8: ['中层硅铝质黄棕壤'], 9: ['中层紫色土'], 10: ['中层紫色沙土'], 11: ['中层酸性紫色土'], 12: ['中层酸性紫色砂土'], 13: ['中层铁质黄壤'], 14: ['中层铁铝质黄壤'], 15: ['中层黄色石灰土'], 16: ['中层黑色石灰土'], 17: ['中白善泥'], 18: ['中白胶泥'], 19: ['冷水田'], 20: ['冷浸田'], 21: ['冷砂土'], 22: ['冷砂田'], 23: ['厚层次生黄色石灰土'], 24: ['厚层淋溶黄色石灰土'], 25: ['厚层硅质黄壤'], 26: ['厚层硅铁质黄壤'], 27: ['厚层硅铁质黄壤性土'], 28: ['厚层硅铁质黄棕壤'], 29: ['厚层硅铝质黄壤'], 30: ['厚层硅铝质黄壤性土'], 31: ['厚层硅铝质黄棕壤'], 32: ['厚层酸性紫色土'], 33: ['厚层酸性紫色砂土'], 34: ['厚层铁质黄壤'], 35: ['厚层铁铝质黄壤'], 36: ['厚层黄色石灰土'], 37: ['厚层黑色石灰土'], 38: ['大土泥'], 39: ['大泥'], 40: ['大泥土'], 41: ['大眼泥'], 42: ['寡黄泥'], 43: ['小土泥'], 44: ['小黄泥'], 45: ['小黄泥土'], 46: ['岩泥'], 47: ['岩灰泥'], 48: ['干鸭屎泥'], 49: ['扁砂土'], 50: ['扁砂泥'], 51: ['死黄泥'], 52: ['死黄泥土'], 53: ['浅脚烂泥田'], 54: ['浅血泥田'], 55: ['深脚烂泥田'], 56: ['湿鸭屎泥'], 57: ['潮泥'], 58: ['潮砂土'], 59: ['潮砂泥'], 60: ['火石砂土'], 61: ['灰泡土'], 62: ['灰泡黄泥土'], 63: ['烂锈田'], 64: ['热鸭屎泥'], 65: ['煤锈田'], 66: ['熟白善泥'], 67: ['熟白胶泥'], 68: ['白云砂土'], 69: ['白大泥'], 70: ['白砂土'], 71: ['白胶泥土'], 72: ['石喳子土'], 73: ['砾石紫泥土'], 74: ['紫油泥'], 75: ['紫油砂土'], 76: ['紫泥'], 77: ['紫砂土'], 78: ['紫砂泥'], 79: ['紫色土'], 80: ['紫色大土'], 81: ['羊肝泥'], 82: ['羊肝石土'], 83: ['苦大泥'], 84: ['苦鸭屎泥'], 85: ['薄层次生黄色石灰土'], 86: ['薄层淋溶黄色石灰土'], 87: ['薄层硅质黄壤'], 88: ['薄层硅质黄壤性土'], 89: ['薄层硅铁质黄壤'], 90: ['薄层硅铁质黄壤性土'], 91: ['薄层硅铝质黄壤'], 92: ['薄层硅铝质黄壤性土'], 93: ['薄层酸性紫色土'], 94: ['薄层铁铝质黄壤'], 95: ['薄层黄色石灰土'], 96: ['薄层黑色石灰土'], 97: ['血砂泥土'], 98: ['豆办泥土'], 99: ['豆办砂泥'], 100: ['豆面泥土'], 101: ['轻白善泥'], 102: ['轻白胶泥'], 103: ['重白善泥'], 104: ['重白胶泥'], 105: ['锈水田'], 106: ['黄大泥'], 107: ['黄油砂土'], 108: ['黄油砂泥'], 109: ['黄泥'], 110: ['黄泥土'], 111: ['黄灰泡土'], 112: ['黄石砂田'], 113: ['黄砂土'], 114: ['黄砂泥'], 115: ['黄胶泥'], 116: ['黄胶泥土'], 117: ['黑潮泥'], 118: ['黑色油砂石'], 119: ['龙凤大眼泥']}

In [28]:
result_dict = {}
for one_key in tz_dict:
    result_dict[one_key] = {'TZ':tz_dict[one_key][0],'TL':soil_dict[tz_dict[one_key][0]]['TL'],'TS':soil_dict[tz_dict[one_key][0]]['TS'],'YL':soil_dict[tz_dict[one_key][0]]['YL']}

In [29]:
result_dict

{0: {'TZ': '中层次生黄色石灰土', 'TL': '石灰土', 'TS': '次生黄色石灰土', 'YL': '黄色石灰土'},
 1: {'TZ': '中层淋溶黄色石灰土', 'TL': '石灰土', 'TS': '淋溶黄色石灰石', 'YL': '黄色石灰土'},
 2: {'TZ': '中层硅质黄壤', 'TL': '黄壤', 'TS': '硅质黄壤', 'YL': '黄壤'},
 3: {'TZ': '中层硅铁质黄壤', 'TL': '黄壤', 'TS': '硅铁质黄壤', 'YL': '黄壤'},
 4: {'TZ': '中层硅铁质黄壤性土', 'TL': '黄壤', 'TS': '硅铁质黄壤性土', 'YL': '黄壤性土'},
 5: {'TZ': '中层硅铁质黄棕壤', 'TL': '山地黄棕壤', 'TS': '硅铁质黄棕壤', 'YL': '山地黄壤'},
 6: {'TZ': '中层硅铝质黄壤', 'TL': '黄壤', 'TS': '硅铝质黄壤', 'YL': '黄壤'},
 7: {'TZ': '中层硅铝质黄壤性土', 'TL': '黄壤', 'TS': '硅铝质黄壤性土', 'YL': '黄壤性土'},
 8: {'TZ': '中层硅铝质黄棕壤', 'TL': '山地黄棕壤', 'TS': '硅铝质黄棕壤', 'YL': '山地黄壤'},
 9: {'TZ': '中层紫色土', 'TL': '紫色土', 'TS': '紫色土', 'YL': '钙质和中性紫色土'},
 10: {'TZ': '中层紫色沙土', 'TL': '紫色土', 'TS': '紫色沙土', 'YL': '钙质和中性紫色土'},
 11: {'TZ': '中层酸性紫色土', 'TL': '紫色土', 'TS': '酸性紫色土', 'YL': '酸性紫色土'},
 12: {'TZ': '中层酸性紫色砂土', 'TL': '紫色土', 'TS': '酸性紫色砂土', 'YL': '酸性紫色土'},
 13: {'TZ': '中层铁质黄壤', 'TL': '黄壤', 'TS': '铁质黄壤', 'YL': '黄壤'},
 14: {'TZ': '中层铁铝质黄壤', 'TL': '黄壤', 'TS': '铁铝质黄壤', 'YL': '黄壤'},
 15: {'TZ

In [30]:
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_soiltype_result\feature_soiltype_result_ky.gdb\prediction_sioltype_RandomForestEntr_20240411_reclass_smooth'
out_polygon_path = r'D:\ArcGISProjects\workspace\shbyq\soil_type_result.gdb\soiltype_ky\prediction_sioltype_RandomForestEntr_20240411_reclass_smooth'
field_name = 'Value'

In [31]:
# 栅格转面
raster_to_polygon(raster_path, out_polygon_path, field_name)

In [40]:
code_block =f"""
def get_soil_class(soil_code):
    soil_dict = {result_dict}
    TZ = soil_dict[soil_code]['TZ']
    TL = soil_dict[soil_code]['TL']
    YL = soil_dict[soil_code]['YL']
    TS = soil_dict[soil_code]['TS']
    result_list = [TZ,TS,YL,TL]
    return result_list[0]
"""
field_name = 'TZ'

In [33]:
expression = "get_soil_class(!gridcode!)"

In [34]:
# 增加字段
for one_field in ['TL','YL','TS','TZ']:
    add_field(out_polygon_path, one_field, 'TEXT')

In [41]:
# 计算字段
arcpy.CalculateField_management(in_table=out_polygon_path, field=field_name, expression=expression, expression_type="PYTHON3", code_block=code_block)
    

<Result 'D:\\ArcGISProjects\\workspace\\shbyq\\soil_type_result.gdb\\soiltype_ky\\prediction_sioltype_RandomForestEntr_20240411_reclass_smooth'>